# Transformer-based language models

<a target="_blank" href="https://colab.research.google.com/github/jaspock/me/blob/main/docs/materials/transformers/assets/notebooks/nerbert.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Code written by Juan Antonio Pérez in 2024.

This notebook presents 

**Exercise**: use spm to tokenize the data 

- Nos ahorramos el padding en make_batch
- Comentar sobre desconocidas y UNK en train y test
- Hacer versión corta con MultiHeadAttention y LN de PyTorch
- to-do: check that original multiheadattention behaves similar to the simpler one
- mover transformer a notebook y poner un main que ejecuta uno muy pequeño inicializado aleatoriamente
- wget github y %run 'note.ipynb' para ejecutarlo en celda de notebook
- hacer una clase comun a encoder y decoder (solo cambia la mascara), TransformerModule, 
- EncoderOnlyTranformer, DecoderOnlyTransformer
- poner set_seed en los otros notebooks y evaluarlo
- añadir a NER el lr scheduler
- Ejercicio: que funcione con minibatches en inferencia
- Repasar el main del BERT para hacerlo como aquí
- usar no_grad en todas la evaluaciones
- avisar que no usaremos prefix tuning o como se llame (supongo que hay que usarla tb durante el training)
- ejercicio: top-p sampling
- sustituir !pip install por %pip en todos

In [7]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


## Mini-batch preparation

In [8]:
import torch

def make_batch(corpus, word_index, max_len, batch_size, device):

    tokens = corpus.split()
    token_indices = [word_index.get(token, word_index['[UNK]']) for token in tokens]
    n_tokens = len(token_indices)  # number of tokens in the corpus
    batch_token_length = batch_size * max_len  # the total number of tokens in a batch
    assert n_tokens >= batch_token_length, f'Short corpus ({n_tokens} tokens), must be at least {batch_length} tokens long'

    while True:
        input_batch, output_batch = [], []
        
        for _ in range(batch_size):
            start_index = random.randint(0, n_tokens - 1)  # random start
            end_index = start_index + max_len
            input_seq = token_indices[start_index:end_index]
            if end_index > n_tokens:
                input_seq += token_indices[:end_index - n_tokens]
            
            # output is the same as input, except shifted one token to the right
            output_seq = input_seq[1:] + [token_indices[end_index % n_tokens]]

            input_batch.append(input_seq)
            output_batch.append(output_seq)

        yield torch.LongTensor(input_batch).to(device), torch.LongTensor(output_batch).to(device)

## Import our transformer code

In [9]:
%%capture

import os
colab = bool(os.getenv("COLAB_RELEASE_TAG"))
if not os.path.isfile('transformer.ipynb') and colab:
    %pip install wget
    %wget https://raw.githubusercontent.com/jaspock/minGPT/master/transformer.ipynb

%pip install nbformat
%run './transformer.ipynb'

set_seed(42)

## Corpus preprocessing

In [10]:


corpus = """
TEODORO:	
Fuese. ¿Quién pensó jamás	
de mujer tan noble y cuerda	
este arrojarse tan presto	
a dar su amor a entender?	
Pero también puede ser	
que yo me engañase en esto.	
Mas no me ha dicho jamás,	
ni a lo menos se me acuerda:	
«Pues ¿qué importa que se pierda,	
si se puede perder más?»	
Perder más... Bien puede ser	
por la mujer que decía...	
Mas todo es bachillería,	
y ella es la misma mujer.	
Aunque no, que la Condesa	
es tan discreta y tan varia	
que es la cosa más contraria	
de la ambición que profesa.	
Sírvenla príncipes hoy	
en Nápoles. ¿Qué no puedo	
ser su esclavo? Tengo miedo,	
que en grande peligro estoy.	
Ella sabe que a Marcela	
sirvo, pues aquí ha fundado	
el engaño y me ha burlado.	
Pero en vano se recela	
mi temor, porque jamás	
burlando salen colores.	
¿Y el decir con mil temores	
que se puede perder más?	
¿Qué rosa al llorar la Aurora	
hizo de las hojas ojos,	
abriendo los labios rojos	
con risa a ver cómo llora	
como ella los puso en mí,	
bañada en púrpura y grana,	
o qué pálida manzana	
se esmaltó de carmesí?	
Lo que veo y lo que escucho	
yo lo juzgo, o estoy loco,	
para ser de veras, poco,	
y para de burlas, mucho.	
Mas teneos, pensamiento,	
que os vais ya tras la grandeza,	
aunque si digo belleza	
bien sabéis vos que no miento,	
que es bellísima Diana	
y es discreción sin igual.

MARCELA:	
¿Puedo hablarte?

TEODORO:	
Ocasión tal	
mil imposibles allana,	
que por ti, Marcela mía,	
la muerte me es agradable.

MARCELA:	
Como yo te vea y hable,	
dos mil vidas perdería.	
Estuve esperando el día	
como el pajarillo solo	
y, cuando vi que en el polo	
que Apolo más presto dora	
le despertaba la Aurora,	
dije: «Yo veré mi Apolo.»	
Grandes cosas han pasado,	
que no se quiso acostar	
la Condesa hasta dejar	
satisfecho su cuidado;	
amigas que han envidiado	
mi dicha con deslealtad	
le han contado la verdad,	
que entre quien sirve, aunque veas	
que hay amistad, no la creas,	
porque es fingida amistad.	
Todo lo sabe en efeto,	
que si es Diana la luna,	
siempre a quien ama importuna,	
salió y vio nuestro secreto;	
pero será, te prometo,	
para mayor bien, Teodoro,	
que del honesto decoro	
con que tratas de casarte	
le di parte, y dije aparte	
cuán tiernamente te adoro;	
tus prendas le encarecí,	
tu estilo, tu gentileza,	
y ella entonces su grandeza	
mostró tan piadosa en mí,	
que se alegró de que en ti	
hubiese los ojos puesto	
y de casarnos muy presto	
palabra también me dio,	
luego que de mí entendió	
que era tu amor tan honesto.	
Yo pensé que se enojara	
y la casa revolviera,	
que a los dos nos despidiera	
y a los demás castigara,	
mas su sangre ilustre y clara	
y aquel ingenio en efeto	
tan prudente y tan perfeto	
conoció lo que mereces.	
¡Oh, bien haya, amén mil veces,	
quien sirve a señor discreto!

TEODORO:	
¿Que casarme prometió	
contigo?

MARCELA:	
¿Pones duda	
que a su ilustre sangre acuda?

TEODORO:	
Mi ignorancia me engañó
"""

word_list = list(set(corpus.split()))
word_index = {'[PAD]': 0, '[UNK]': 1, '[EOS]': 2}
special_tokens= len(word_index) 
for i, w in enumerate(word_list):
    word_index[w] = i + special_tokens
index_word = {i: w for i, w in enumerate(word_index)}
vocab_size = len(word_index)
print("vocab size: %d" % vocab_size)



vocab size: 315


## Model training

In [11]:
n_layer = 2
n_head = 2
n_embd =  64
embd_pdrop = 0.1
resid_pdrop = 0.1
attn_pdrop = 0.1
batch_size = 4
max_len = 32
training_steps = 1000
eval_steps = 100
lr = 0.001

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = DecoderTransformer(n_embd=n_embd, n_head=n_head, n_layer=n_layer, vocab_size=vocab_size,  
                max_len=max_len, embd_pdrop=embd_pdrop, attn_pdrop=attn_pdrop, resid_pdrop=resid_pdrop)
model.to(device)

criterion = nn.CrossEntropyLoss(ignore_index=0)  # not needed here since we are not padding inputs
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.OneCycleLR(optimizer, max_lr=lr, steps_per_epoch=training_steps, epochs=1, anneal_strategy='cos')

model.train()
step = 0

for inputs, outputs in make_batch(corpus, word_index, max_len, batch_size, device):
    optimizer.zero_grad()
    logits = model(inputs)
    loss = criterion(logits.view(-1,logits.size(-1)), outputs.view(-1)) 
    if i % eval_steps == 0:
        print(f'Step [{i}/{training_steps}], loss: {loss.item():.4f}')
    loss.backward()
    optimizer.step()
    scheduler.step()
    step = step + 1
    if (step==training_steps):
        break

number of parameters: 0.12M
Step     1, loss 5.754934
Step   100, loss 5.313444
Step   200, loss 3.775151
Step   300, loss 1.498896
Step   400, loss 0.850160
Step   500, loss 0.571959
Step   600, loss 0.430279
Step   700, loss 0.237594
Step   800, loss 0.189764
Step   900, loss 0.180697
Step  1000, loss 0.219462


## Model evaluation


In [12]:

def generate_text(model, prompt, word_index, index_word, max_len, device):
    words = prompt.split()
    input_ids = [word_index.get(word, word_index['[UNK]']) for word in words]
    input = torch.LongTensor(input_ids).view(1, -1).to(device)  # add batch dimension

    with torch.no_grad():
        for _ in range(max_len - len(input_ids)):
            output = model(input)
            last_token_logits = output[0, -1, :]
            predicted_id = torch.argmax(last_token_logits, dim=-1).item()
            input = torch.cat([input, torch.LongTensor([predicted_id]).view(1,-1).to(device)], dim=1)
            predicted_word = index_word[predicted_id]
            words.append(predicted_word)
            if predicted_word == '[EOS]':
                break

    return ' '.join(words)

model.eval()
prompt = """TEODORO: 
Cosas como estas
son la cartilla, señora,	
de quien ama y quien desea.

MARCELA:"""
generated_text = generate_text(model, prompt, word_index, index_word, max_len, device)
print(generated_text)


TEODORO: Cosas como estas son la cartilla, señora, de quien ama y quien desea. MARCELA: ¿Pones duda que a su ilustre sangre acuda? TEODORO: Mi ignorancia me engañó TEODORO: Fuese. ¿Quién pensó
